In [2]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split

# Define the directory paths
base_dir = 'colored_images'
categories = ['No_DR', 'Mild', 'Moderate', 'Severe', 'Proliferate_DR']

# Prepare lists to hold image data and labels
data = []
labels = []

# Loop through each category folder
for idx, category in enumerate(categories):
    folder_path = os.path.join(base_dir, category)
    for img_file in os.listdir(folder_path):
        try:
            # Read the image
            img_path = os.path.join(folder_path, img_file)
            image = cv2.imread(img_path)
            # Resize the image to 224x224 pixels
            image = cv2.resize(image, (224, 224))
            # Append the image and its corresponding label to the lists
            data.append(image)
            labels.append(idx)  # Label is the index of the category
        except Exception as e:
            print(f"Error loading image {img_path}: {e}")

# Convert lists to numpy arrays
data = np.array(data) / 255.0  # Normalize pixel values
labels = np.array(labels)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

print(f"Training set: {X_train.shape}, {y_train.shape}")
print(f"Testing set: {X_test.shape}, {y_test.shape}")


Training set: (2929, 224, 224, 3), (2929,)
Testing set: (733, 224, 224, 3), (733,)


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Define the CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(5, activation='softmax')  # 5 classes for different DR stages
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=32)

# Save the model
model.save('dr_model.h5')


Epoch 1/10


: 

Epoch 1/10
92/92 [==============================] - 47s 492ms/step - loss: 0.9247 - accuracy: 0.6743 - val_loss: 0.7825 - val_accuracy: 0.7271
Epoch 2/10
92/92 [==============================] - 43s 471ms/step - loss: 0.7998 - accuracy: 0.7091 - val_loss: 0.7687 - val_accuracy: 0.7162
Epoch 3/10
92/92 [==============================] - 43s 471ms/step - loss: 0.7843 - accuracy: 0.7197 - val_loss: 0.7377 - val_accuracy: 0.7367
Epoch 4/10
92/92 [==============================] - 44s 475ms/step - loss: 0.7648 - accuracy: 0.7272 - val_loss: 0.7327 - val_accuracy: 0.7271
Epoch 5/10
92/92 [==============================] - 44s 478ms/step - loss: 0.7499 - accuracy: 0.7289 - val_loss: 0.7316 - val_accuracy: 0.7285
Epoch 6/10
92/92 [==============================] - 43s 469ms/step - loss: 0.7326 - accuracy: 0.7303 - val_loss: 0.7411 - val_accuracy: 0.7271
Epoch 7/10
92/92 [==============================] - 43s 468ms/step - loss: 0.7102 - accuracy: 0.7470 - val_loss: 0.7314 - val_accuracy: 0.7203
Epoch 8/10
92/92 [==============================] - 43s 467ms/step - loss: 0.6699 - accuracy: 0.7559 - val_loss: 0.7004 - val_accuracy: 0.7422
Epoch 9/10
92/92 [==============================] - 43s 467ms/step - loss: 0.6446 - accuracy: 0.7675 - val_loss: 0.6768 - val_accuracy: 0.7531
Epoch 10/10
92/92 [==============================] - 43s 466ms/step - loss: 0.5839 - accuracy: 0.7798 - val_loss: 0.7274 - val_accuracy: 0.7353
/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(

In [1]:
import tensorflow as tf

# Convert the model to TensorFlow Lite format
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the converted model
with open('dr_model.tflite', 'wb') as f:
    f.write(tflite_model)


NameError: name 'model' is not defined